In [1]:
from pyspark.sql import SparkSession

# New API
spark_session = SparkSession\
        .builder\
        .master("spark://group101:7077") \
        .appName("simple_example")\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext

In [12]:
fasta_file = spark_context.textFile('file:///home/ubuntu/data/pdb_seqres.txt')

['>101m_A mol:protein length:154  MYOGLOBIN',
 'MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDRVKHLKTEAEMKASEDLKKHGVTVLTALGAILKKKGHHEAELKPLAQSHATKHKIPIKYLEFISEAIIHVLHSRHPGNFGADAQGAMNKALELFRKDIAAKYKELGYQG',
 '>102l_A mol:protein length:165  T4 LYSOZYME',
 'MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAAKSELDKAIGRNTNGVITKDEAEKLFNQDVDAAVRGILRNAKLKPVYDSLDAVRRAALINMVFQMGETGVAGFTNSLRMLQQKRWDEAAVNLAKSRWYNQTPNRAKRVITTFRTGTWDAYKNL',
 '>102m_A mol:protein length:154  MYOGLOBIN']

In [13]:
# Split the fasta file into headers and amino acid sequences
headers = fasta_file.filter(lambda x: ">" in x)
sequences = fasta_file.filter(lambda x: ">" not in x)

In [6]:
headers.take(5)

['>101m_A mol:protein length:154  MYOGLOBIN',
 '>102l_A mol:protein length:165  T4 LYSOZYME',
 '>102m_A mol:protein length:154  MYOGLOBIN',
 '>103l_A mol:protein length:167  T4 LYSOZYME',
 '>103m_A mol:protein length:154  MYOGLOBIN']

In [7]:
headers.count()

446948

In [8]:
sequences.count()

In [9]:
sequences.take(5)

['MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDRVKHLKTEAEMKASEDLKKHGVTVLTALGAILKKKGHHEAELKPLAQSHATKHKIPIKYLEFISEAIIHVLHSRHPGNFGADAQGAMNKALELFRKDIAAKYKELGYQG',
 'MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAAKSELDKAIGRNTNGVITKDEAEKLFNQDVDAAVRGILRNAKLKPVYDSLDAVRRAALINMVFQMGETGVAGFTNSLRMLQQKRWDEAAVNLAKSRWYNQTPNRAKRVITTFRTGTWDAYKNL',
 'MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDRFKHLKTEAEMKASEDLKKAGVTVLTALGAILKKKGHHEAELKPLAQSHATKHKIPIKYLEFISEAIIHVLHSRHPGNFGADAQGAMNKALELFRKDIAAKYKELGYQG',
 'MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNSLDAAKSELDKAIGRNTNGVITKDEAEKLFNQDVDAAVRGILRNAKLKPVYDSLDAVRRAALINMVFQMGETGVAGFTNSLRMLQQKRWDEAAVNLAKSRWYNQTPNRAKRVITTFRTGTWDAYKNL',
 'MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDRFKHLKTEAEMKASEDLKKAGVTVLTALGAILKKKGHHEAELKPLAQSHATKHKIPIKYLEFISEAIIHVLHSRHPGNFGADAQGAMNKALELFRKDIAAKYKELGYQG']

In [18]:
split=sequences.map(lambda seq: seq.split(""))

In [20]:
#counts=sequences.flatMap(lambda x:[(c,1) for c in x]).reduceByKey(add)

NameError: name 'add' is not defined

In [21]:
# function that 
def count_occur(str):
   uniq = set(str)
   li = list(str)
   dict = {}
   for key in uniq:
       dict[key] = str.count(key)
   return dict

In [26]:
# Counting the occurence of each character in each protein
sequences.map(lambda x: count_occur(x)).take(2)

[{'A': 17,
  'D': 6,
  'E': 14,
  'F': 5,
  'G': 11,
  'H': 12,
  'I': 9,
  'K': 19,
  'L': 18,
  'M': 3,
  'N': 2,
  'P': 4,
  'Q': 5,
  'R': 4,
  'S': 6,
  'T': 5,
  'V': 9,
  'W': 2,
  'Y': 3},
 {'A': 17,
  'D': 10,
  'E': 8,
  'F': 5,
  'G': 11,
  'H': 1,
  'I': 10,
  'K': 13,
  'L': 16,
  'M': 5,
  'N': 12,
  'P': 3,
  'Q': 5,
  'R': 13,
  'S': 6,
  'T': 12,
  'V': 9,
  'W': 3,
  'Y': 6}]

In [30]:
# Counting the length of each sequence
lengths = sequences.map(lambda x:len(x))
lengths.zipWithIndex().min()

(1, 422820)

In [27]:
# This is with the index of each line
sequences.map(lambda x: count_occur(x)).zipWithIndex().take(5)

[({'A': 17,
   'D': 6,
   'E': 14,
   'F': 5,
   'G': 11,
   'H': 12,
   'I': 9,
   'K': 19,
   'L': 18,
   'M': 3,
   'N': 2,
   'P': 4,
   'Q': 5,
   'R': 4,
   'S': 6,
   'T': 5,
   'V': 9,
   'W': 2,
   'Y': 3},
  0),
 ({'A': 17,
   'D': 10,
   'E': 8,
   'F': 5,
   'G': 11,
   'H': 1,
   'I': 10,
   'K': 13,
   'L': 16,
   'M': 5,
   'N': 12,
   'P': 3,
   'Q': 5,
   'R': 13,
   'S': 6,
   'T': 12,
   'V': 9,
   'W': 3,
   'Y': 6},
  1),
 ({'A': 18,
   'D': 6,
   'E': 14,
   'F': 6,
   'G': 11,
   'H': 11,
   'I': 9,
   'K': 19,
   'L': 18,
   'M': 3,
   'N': 2,
   'P': 4,
   'Q': 5,
   'R': 4,
   'S': 6,
   'T': 5,
   'V': 8,
   'W': 2,
   'Y': 3},
  2),
 ({'A': 16,
   'D': 11,
   'E': 8,
   'F': 5,
   'G': 11,
   'H': 1,
   'I': 10,
   'K': 13,
   'L': 17,
   'M': 5,
   'N': 12,
   'P': 3,
   'Q': 5,
   'R': 13,
   'S': 7,
   'T': 12,
   'V': 9,
   'W': 3,
   'Y': 6},
  3),
 ({'A': 18,
   'D': 6,
   'E': 14,
   'F': 6,
   'G': 11,
   'H': 11,
   'I': 9,
   'K': 19,
   'L': 18,
 

In [ ]:
# release the cores for another application!
spark_context.stop()